In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/10-701 Project/maestro-v3.0.0'

Mounted at /content/drive


In [ ]:
!pip install mido
!pip install tqdm
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.1 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
from mido import MidiFile
mid = MidiFile(path + '/' + '2004' + '/' + 'MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi')
for i, track in enumerate(mid.tracks):
    print(f'Track {i}: {track.name}')
    # for msg in track:
    #     print(msg)

Track 0: 
Track 1: 


In [ ]:
ticks_per_beat = mid.ticks_per_beat
tempo = None
for msg in mid.tracks[0]:
    if msg.type == 'set_tempo':
        tempo = msg.tempo
        break
print('tempo', tempo)
tick_duration = tempo / (1_000_000 * ticks_per_beat)
print('tick_duration', tick_duration)
print('1/tick_duration', 1/tick_duration)

tempo 500000
tick_duration 0.0010416666666666667
1/tick_duration 960.0


In [ ]:
copy_msg = None
orig_msg = None
for msg in mid.tracks[1]:
  #if msg.type == 'note_on':
  orig_msg = msg
  copy_msg = msg.copy(time=1)
  input()
  print(copy_msg)
  #break

print(orig_msg)
print(copy_msg)




program_change channel=0 program=0 time=1

control_change channel=0 control=64 value=110 time=1

control_change channel=0 control=67 value=117 time=1

control_change channel=0 control=64 value=114 time=1

control_change channel=0 control=64 value=118 time=1

control_change channel=0 control=67 value=121 time=1

control_change channel=0 control=64 value=122 time=1

note_on channel=0 note=71 velocity=60 time=1

control_change channel=0 control=67 value=117 time=1

control_change channel=0 control=67 value=113 time=1

note_on channel=0 note=71 velocity=0 time=1

control_change channel=0 control=64 value=111 time=1

control_change channel=0 control=64 value=93 time=1

control_change channel=0 control=64 value=76 time=1

note_on channel=0 note=55 velocity=44 time=1

note_on channel=0 note=71 velocity=54 time=1


KeyboardInterrupt: Interrupted by user

In [ ]:
import numpy as np
trk = iter(track)
midi_df = np.empty((0,2))
for msg in track:
  if msg.type == 'note_on':
    midi_df = np.append(midi_df,np.tile([msg.note, msg.velocity],(msg.time,1)),axis=0)
midi_df

array([[71., 60.],
       [71., 60.],
       [71., 60.],
       ...,
       [62.,  0.],
       [62.,  0.],
       [62.,  0.]])

In [ ]:
from mido import MidiFile
import numpy as np

from torch.utils.data import Dataset, DataLoader
import torch
import os
import math

TRAIN_SAMPLING_RATE = 10    # number of datapoints we are extracting from each second


class MidiDataset(Dataset):
  def __init__(self,
               root_dir,
               stride,
               window_size,
               train_sampling_rate
               ):
    self.root_dir = root_dir
    self.stride = stride
    self.train_sampling_rate = train_sampling_rate
    self.window_size = window_size
    # self.resolution is no longer fixed, it will be calculated per-file.
    self._build_index()

  def obtain_midi_sampling_frequency(self, mid_file):
    """
    Calculates the number of MIDI ticks per second for a given file.
    """
    ticks_per_beat = mid_file.ticks_per_beat

    tempo = None

    # Look for a tempo change message in the first track
    if mid_file.tracks:
        for msg in mid_file.tracks[0]:
            if msg.type == 'set_tempo':
                tempo = msg.tempo
                break

    # (microseconds per beat) / (microseconds per sec * ticks per beat) = seconds per tick
    tick_duration_sec = tempo / (1_000_000 * ticks_per_beat)

    # (ticks per second)
    file_sampling_rate = 1.0 / tick_duration_sec
    return file_sampling_rate

  def resolution_from_frequency(self, file_sampling_rate, train_sampling_rate):
    """
    Calculates the integer step size (resolution) required to slice the
    tick-based array at the desired training sample rate.

    (ticks / sec) / (samples / sec) = (ticks / sample)
    """

    resolution = file_sampling_rate / train_sampling_rate

    # Must return an integer for array slicing, and it must be at least 1.
    return round(resolution)


  def _build_index(self):
    midi_files = []
    for dirpath,_,filenames in os.walk(self.root_dir):
      for filename in filenames:
        if filename.lower().endswith(".midi"):
          midi_files.append(os.path.join(dirpath, filename))

    # for f in midi_files[:10]:
    #   print(f)
    total_steps = 0
    self.sample_list = []

    for f in midi_files[:50]:
      mid = MidiFile(f)
      file_ticks_per_sec = self.obtain_midi_sampling_frequency(mid)

      resolution = self.resolution_from_frequency(
        file_ticks_per_sec,
        self.train_sampling_rate
      )

      track = mid.tracks[1]
      midi_df = np.empty((0,2))

      for msg in track:
        if msg.type == 'note_on':
          # df containing all quantized notes in a file
          midi_df = np.append(midi_df,np.tile([msg.note, msg.velocity],(msg.time,1)),axis=0)
      #print(midi_df)
      midi_df = midi_df[::self.resolution,:]

      steps = (midi_df.shape[0] - self.window_size)/self.stride
      steps = math.floor(steps)+1

      print(steps)
      total_steps += steps

      for step in range(steps):
        sample = midi_df[step*self.stride:step*self.stride+self.window_size]
        self.sample_list.append(sample)

    print(total_steps)
  def __len__(self):
    return(len(self.sample_list))
    pass
  def __getitem__(self, idx):
    return torch.from_numpy(self.sample_list[idx])
    pass


In [ ]:
MidiDataset = MidiDataset(root_dir=path,
                          stride=960,
                          window_size=960*10,
                          resolution=1)
len(MidiDataset)




427
566
388
254
160
333


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm

class SimpleRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size):
    super(SimpleRNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(
      input_size=input_size,
      hidden_size=hidden_size,
      num_layers=num_layers,
      batch_first=True
    )
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    # Shape: (num_layers, batch_size, hidden_size)
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

    # rnn_out shape: (batch_size, window_size, hidden_size)
    rnn_out, _ = self.rnn(x, h0) # We only pass in h0

    # logits shape: (batch_size, window_size, output_size)
    logits = self.fc(rnn_out)

    return logits

In [ ]:
STRIDE = 960              # one midi second
WINDOW_SIZE = STRIDE*10   # 10 seconds
RESOLUTION = 96
BATCH_SIZE = 4
NUM_EPOCHS = 20
LEARNING_RATE = 1e-3

# model parameters
INPUT_SIZE = 1
HIDDEN_SIZE = 512
NUM_LAYERS = 1
OUTPUT_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device is", device)

dataloader = MidiDataset(root_dir=path,
                          stride=STRIDE,
                          window_size=WINDOW_SIZE,
                          resolution=RESOLUTION)

len(dataloader)

model = SimpleRNN(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE).to(device)


criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(NUM_EPOCHS):
  epoch_loss = 0
  for i, batch in tqdm(enumerate(dataloader)):
    batch = batch.to(device)

    # y: notes
    labels = batch[:, :, 0].long()

    # X: velocities
    inputs = batch[:, :, 1].float()

    # normalize velocities to [0,1]
    inputs = inputs / 127.0

    inputs = inputs.unsqueeze(-1)

    model.train()
    optimizer.zero_grad()

    logits = model(inputs)

    loss = criterion(logits.view(-1, OUTPUT_SIZE), labels.view(-1))

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
  avg_epoch_loss = epoch_loss / len(dataloader)
  print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_epoch_loss:.4f}")






device is cpu
427
566
388
254
160
333
423
363
144
237
3295


TypeError: 'module' object is not callable. Did you mean: 'tqdm.tqdm(...)'?